In [1]:

#importing the needed libraries for modeling
#Sequential api
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [2]:
#Loading the dataset
df=pd.read_csv(r"heart.csv")


In [3]:
print(df.to_string())

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  ca  thal  target
0     63    1   3       145   233    1        0      150      0      2.3      0   0     1       1
1     37    1   2       130   250    0        1      187      0      3.5      0   0     2       1
2     41    0   1       130   204    0        0      172      0      1.4      2   0     2       1
3     56    1   1       120   236    0        1      178      0      0.8      2   0     2       1
4     57    0   0       120   354    0        1      163      1      0.6      2   0     2       1
5     57    1   0       140   192    0        1      148      0      0.4      1   0     1       1
6     56    0   1       140   294    0        0      153      0      1.3      1   0     2       1
7     44    1   1       120   263    0        1      173      0      0.0      2   0     3       1
8     52    1   2       172   199    1        1      162      0      0.5      2   0     3       1
9     57    1   2   

In [30]:
#Handling the missing values
df = df.dropna()

In [31]:
df.drop(columns=['Age','Sex'],inplace=True)


In [32]:
#converting the string values to int values
lab = LabelEncoder()

In [33]:
df['ChestPainType'] = lab.fit_transform(df['ChestPainType'])
df['ExerciseAngina'] = lab.fit_transform(df['ExerciseAngina'])
df['ST_Slope'] = lab.fit_transform(df['ST_Slope'])
df['RestingECG'] = lab.fit_transform(df['RestingECG'])
df=df.astype('float32')

In [34]:
x = df.iloc[:,[0,1,2,3,4,5,6,7,8]].values
y = df.iloc[:,-1].values

In [35]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y)

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=0,test_size=0.33)

In [37]:
y

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], shape=(918, 2))

In [38]:
st = StandardScaler()

In [39]:
x_train = st.fit_transform(x_train)
x_test = st.fit_transform(x_test)

In [40]:
print(x.shape)
print(y.shape)

(918, 9)
(918, 2)


In [41]:
model = Sequential()   
model.add(Dense(126,input_shape=(9,),activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(18,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer = Adam(learning_rate=0.001),loss = 'binary_crossentropy',metrics = ['accuracy'])




c:\Users\abhij\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
#compile the model
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 126)            │         1,260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 18)             │           594 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 4)              │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │            10 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,148 (47.45 KB)

 Trainable params: 12,148 (47.45 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
#print(model.predict(np.array([[-0.833306,-0.482032,0.696685,-0.527813,-1.573560,1.136740,-0.835999,-0.362645,-0.587665]])))

In [ ]:
#scores=model.evaluate(x_train,y_train)

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 1.7886e-04


In [19]:
#print("%s: %.4f  " %(model.metrics_names[0],scores[0]))

loss: 0.0002  


In [43]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stop=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)
reduce=ReduceLROnPlateau(monitor="val_loss",factor=0.2,patience=5)


In [64]:
model.fit(x_train,y_train,validation_split=0.2,epochs=150,batch_size=32,callbacks=[early_stop,reduce])

Epoch 1/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9992 - loss: 0.0038 - val_accuracy: 1.0000 - val_loss: 0.0086 - learning_rate: 4.0960e-12
Epoch 2/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9970 - loss: 0.0061 - val_accuracy: 1.0000 - val_loss: 0.0086 - learning_rate: 4.0960e-12
Epoch 3/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9996 - loss: 0.0024 - val_accuracy: 1.0000 - val_loss: 0.0086 - learning_rate: 4.0960e-12
Epoch 4/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9937 - loss: 0.0115 - val_accuracy: 1.0000 - val_loss: 0.0086 - learning_rate: 4.0960e-12
Epoch 5/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9970 - loss: 0.0065 - val_accuracy: 1.0000 - val_loss: 0.0086 - learning_rate: 4.0960e-12
Epoch 6/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9994 - loss: 0.0031 - val_accuracy: 1.0000 - val_loss: 0.0086 - learning_rate: 4.0960e-12
Epoch 7/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 

In [67]:
scores=model.evaluate(x_train,y_train)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9980 - loss: 0.0054


In [65]:
y_pred=model.predict(x_test)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


In [66]:
from sklearn.metrics import r2_score
score=r2_score(y_test,y_pred)
print(score*100)


39.92590191600123


In [ ]:
epochs=1000,batch_size=32,verbose=2